In [1]:
from dataset.Norm2dDataset import Norm2dDataset, Norm2dParameter
from sklearn.linear_model import LogisticRegression
from evaluator.visualize import joinplot
import seaborn as sns
import matplotlib.pyplot as plt
from models.logistic_regression import LogisticRegression
import warnings
from DRE.MLBasedDRE import MLBasedDRE
warnings.simplefilter("ignore")

# create dataset

In [2]:
param1 =  {"myu":[10,10], "sigma": [[1,0],[0,1]], "samples": 1000}
param2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "samples": 1000}
dataset = Norm2dDataset((param1, param2))

param = {"penalty":"l2"}
model = LogisticRegression(param)
model.train(dataset.X, dataset.Y)
dataset["rprob"] = model.predict_proba(dataset.X)[:,1].round(1)

# joinplot(dataset, hue = "y")
# joinplot(dataset, hue = "rprob")

In [3]:
dataset["inv_rprob"] = (1 - dataset["rprob"].get_column("rprob")).clip(upper_bound = 2,lower_bound= 0.0001)

In [4]:
(dataset["rprob"].get_column("rprob")/dataset["inv_rprob"].get_column("inv_rprob")).mean()

5000.0

In [5]:
myu = [10,10]
sigma = [[1,0],[0,1]]
size = 10000
data1 = np.random.multivariate_normal(myu, sigma, size)
df1 = pd.DataFrame({"x1":data1[:,0], "x2":data1[:,1]})
df1["y"] = "0"

myu = [0, 0]
sigma = [[1,0],[0 ,1]]
size = 10000
data2 = np.random.multivariate_normal(myu, sigma, size)
df2 = pd.DataFrame({"x1":data2[:,0], "x2":data2[:,1]})
df2["y"] = "1"



NameError: name 'np' is not defined

In [ ]:
params1 =  {"myu":[10,10], "sigma": [[1,0],[0,1]], "size": 1000}
params2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "size": 1000}
create_df(params1, params2);

# modeling

## 必須関数

In [ ]:
def predict(df, x1, x2, y):
    lr = LogisticRegression(penalty='l2')
    lr.fit(df[[x1,x2]], df[y])    
    df["prob"] = lr.predict_proba(df[["x1", "x2"]])[:,0]    
    df["rprob"] = df["prob"].round(1)
        
    # sns.jointplot(data = df,x = x1, y = x2, hue= y)
    sns.jointplot(data = df,x = x1, y = x2, hue= "rprob")
    return df
            

## 完全分離

In [ ]:
predict(df, "x1", "x2", "y");

## 一部重なり

In [ ]:
params1 =  {"myu":[2,2], "sigma": [[1,0],[0,1]], "size": 1000}
params2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "size": 1000}
df = create_df(params1, params2);
predict(df, "x1", "x2", "y");
print()

In [ ]:
params1 =  {"myu":[2,2], "sigma": [[1,0],[0,1]], "size": 50}
params2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "size": 1000}
df = create_df(params1, params2);
predict(df, "x1", "x2", "y");
# print()

In [ ]:
params1 =  {"myu":[0,0], "sigma": [[1,0],[0,1]], "size":1000}
params2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "size": 1000}
df = create_df(params1, params2);
df = predict(df, "x1", "x2", "y");
# print()

In [ ]:
params1 =  {"myu":[2,2], "sigma": [[1,-0.8],[-0.8,1]], "size":2000}
params2 = {"myu":[0,0], "sigma": [[1,0.6],[0.6,1]], "size": 1000}
df = create_df(params1, params2);
df = predict(df, "x1", "x2", "y");
# print()

## random forest

In [ ]:
def predict_random_forest(df, x1, x2, y):
    rf = RandomForestClassifier()
    rf.fit(df[[x1,x2]], df[y])    
    df["prob"] = rf.predict_proba(df[[x1, x2]])[:,0]    
    df["rprob"] = df["prob"].round(1)
        
    sns.jointplot(data = df,x = x1, y = x2, hue= "rprob")
    return df

In [ ]:
from models.random_forest import RandomForest

In [ ]:
params1 =  {"myu":[0,0], "sigma": [[1,0],[0,1]], "size":1000}
params2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "size": 1000}
df = create_df(params1, params2);
df = predict_random_forest(df, "x1", "x2", "y");

In [ ]:
params1 =  {"myu":[2,2], "sigma": [[1,0],[0,1]], "size": 1000}
params2 = {"myu":[0,0], "sigma": [[1,0],[0,1]], "size": 1000}
df = create_df(params1, params2);
predict_random_forest(df, "x1", "x2", "y");
print()

In [ ]:
params1 =  {"myu":[2,2], "sigma": [[1,-0.8],[-0.8,1]], "size":2000}
params2 = {"myu":[0,0], "sigma": [[1,0.6],[0.6,1]], "size": 1000}
df = create_df(params1, params2);
df = predict_random_forest(df, "x1", "x2", "y");
# print()

In [ ]:
params1 =  {"myu":[1,1], "sigma": [[1,-0.8],[-0.8,1]], "size":2000}
params2 = {"myu":[0,0], "sigma": [[1,0.6],[0.6,1]], "size": 1000}
df = create_df(params1, params2);
df = predict_random_forest(df, "x1", "x2", "y");
# print()

## MLP

In [ ]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 10)
        self.fc2 = nn.Linear(10, 1)        
        self.F = F.relu
        self.sigmoid = F.sigmoid

    def forward(self, x):
        x = self.F(self.fc1(x))
        x = self.fc2(x)
        return self.sigmoid(x)
    
def fit(model, x, y):
    criterion = nn.BCELoss().cuda()
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)

    for i in tqdm(range(3000)):
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
def predict(model, x):
    with torch.no_grad():
        pred = model(x)
    return pred

def predict_MLP(df, x1,x2, y):
    model = MLP(2).cuda()
    
    X = torch.from_numpy(df[[x1, x2]].values).cuda().float()
    y = torch.from_numpy(df[["y"]].astype(float).values).cuda().float()
    
    fit(model, X, y)
    
    df["prob"] = predict(model, X).cpu()
    df["rprob"] = df["prob"].round(1)
    sns.jointplot(data = df,x = "x1", y = "x2", hue= "rprob")
    return df


In [ ]:
from models.MLP_

In [ ]:
params1 =  {"myu":[1,1], "sigma": [[0.1,-0.08],[-0.08,0.1]], "size":1000}
params2 = {"myu":[-1,-1], "sigma": [[0.1,0.06],[0.06,0.1]], "size": 1000}
df = create_df(params1, params2);
df = predict_MLP(df, "x1", "x2", "y")

In [ ]:
kyo1 = 0
kyo2 = 0
params1 =  {"myu":[0,0], "sigma": [[0.1,kyo1],[kyo1,0.1]], "size":1000}
params2 = {"myu":[0,0], "sigma": [[0.1,kyo2],[kyo2,0.1]], "size": 1000}
df = create_df(params1, params2);
df = predict_MLP(df, "x1", "x2", "y")

df[df["prob"] < 1E-6] = 1E-6
df[df["prob"] > 1 - 1E-6] = 1 - 1E-6
print((df["prob"]/(1-df["prob"])).mean())

In [ ]:
params1 =  {"myu":[1,1], "sigma": [[1,-0.8],[-0.8,1]], "size":1000}
params2 = {"myu":[0,0], "sigma": [[1,0.6],[0.6,1]], "size": 1000}
df = create_df(params1, params2);
df = predict_random_forest(df, "x1", "x2", "y");

df[df["prob"] < 1E-6] = 1E-6
df[df["prob"] > 1 - 1E-6] = 1 - 1E-6
print((df["prob"]/(1-df["prob"])).mean())

In [ ]:
params1 =  {"myu":[0.05,0.05], "sigma": [[10,0.5],[0.5,10]], "size":1000}
params2 = {"myu":[0,0], "sigma": [[1,0.5],[0.5,1]], "size": 1000}
df = create_df(params1, params2);
df = predict_random_forest(df, "x1", "x2", "y");

df[df["prob"] < 1E-6] = 1E-6
df[df["prob"] > 1 - 1E-6] = 1 - 1E-6
print((df["prob"]/(1-df["prob"])).mean())

In [1]:
import torch

/home/work/.cache/pypoetry/virtualenvs/godaddy-3UyvmeC2-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

False